In [ ]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

### INIT

In [ ]:
import torch

def show_gpu_memory_usage():
    """顯示當前 GPU RAM 使用情況"""
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(device)

        # 獲取記憶體使用情況 (轉換為 GB)
        allocated = torch.cuda.memory_allocated(device) / 1024**3
        reserved = torch.cuda.memory_reserved(device) / 1024**3
        total = torch.cuda.get_device_properties(device).total_memory / 1024**3

        print(f"🖥️  GPU 設備: {device_name}")
        print(f"📊 記憶體使用情況:")
        print(f"   已分配: {allocated:.2f} GB")
        print(f"   已保留: {reserved:.2f} GB")
        print(f"   總容量: {total:.2f} GB")
        print(f"   使用率: {(allocated/total)*100:.1f}%")
        print(f"   保留率: {(reserved/total)*100:.1f}%")

        # 視覺化進度條
        usage_percent = int((allocated/total)*100)
        bar_length = 20
        filled_length = int(bar_length * usage_percent / 100)
        bar = '█' * filled_length + '░' * (bar_length - filled_length)
        print(f"   [{bar}] {usage_percent}%")

    else:
        print("❌ CUDA 不可用，無法檢測 GPU 記憶體")

import torch

def count_parameters(model):
    """
    計算模型的參數數量
    Returns:
      total_params: 包含所有參數
      trainable_params: 只包含 requires_grad=True 的參數
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

import io

def estimate_model_size(model):
    """
    將模型序列化到緩衝區，估算存檔大小（MB）
    """
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size_mb = buffer.getbuffer().nbytes / (1024 ** 2)
    return size_mb

def show_network_status( orig_net, prune_net ):
    print( "原始網路參數統計:\n" )
    model = orig_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

    print( "剪枝網路參數統計:\n" )
    model = prune_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

In [ ]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

In [ ]:
from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

In [ ]:
from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

In [ ]:
from src.lcp.lcp import LCP
lcp = LCP(net, aux_net, dataloader_train)
lcp.init_for_indices()

In [ ]:
from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information.csv' )
# prune_db.initial()

In [ ]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )

In [ ]:
layers = lcp.get_layers_name()
print( layers )
layer_names = []
for name, ch in layers:
    if name.endswith('.conv1') or name.endswith('.conv2'):
        layer_names.append( name )

In [ ]:
lcp.set_prune_db(prune_db)

In [ ]:
from src.lcp.lcpfinetune import LCPFineTune
lcp_finetune = LCPFineTune(
    prune_net = lcp._prune_net,
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg       = cfg,
    optimizer=optimizer,
    parameters=parameters
)
lcp_finetune._setup_logging()

#### Try one layer for channel selection computing for Test SetUp

In [ ]:

for name, ch in layers:
    if name == 'layer2.0.conv2':
        pass
    else:
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")

### Main For Prune + Finetune

##### Set Up for config

In [ ]:
cfg.defrost()

cfg.train.optim.max_iter = 100
cfg.train.do_training = True
cfg.output.print_iter = 10
cfg.eval.iter = 10
cfg.train.batch_size = 1

cfg.freeze()


In [ ]:
from src.util.loss import LCPFinetuneCriterion
lcp_criterion = LCPFinetuneCriterion(
    original_criterion=criterion,  # 原始損失函數實例
    aux_net=lcp._aux_net,  # 包含 aux_loss 方法的實例
    auxiliary_weight=1.0  # 可以調整輔助損失權重
)

#### Load Pruned Network

In [ ]:
from src.lcp.recontruction import LCPReconstruction
lcp_reconstruction = LCPReconstruction(
    prune_db = prune_db,
    pruner = pruner,
    prune_net = lcp._prune_net
)

In [ ]:
lcp_reconstruction.load_checkpoint_with_pruned_net(
    './src/util/checkpoints-test/checkpoint_lcp_finetune_26_layer3.5.conv2.pth'
)
lcp._prune_net = lcp_reconstruction._prune_net
pruned_layers = []
for layer in lcp_reconstruction._pruned_layers:
    if layer not in pruned_layers:
        pruned_layers.append(layer)
for layer in pruned_layers:
    lcp.prune_layer(
        layer_name   = layer,
        discard_rate = None,
    )

##### Main code for LCP Algorithm 1

In [ ]:
# for idx, layer_name in enumerate(layer_names):
#     if layer_name == 'layer3.0.conv1' or layer_name == 'layer3.1.conv1' or layer_name == 'layer3.2.conv1' or layer_name == 'layer3.3.conv1' or layer_name == 'layer3.4.conv1' or layer_name == 'layer3.5.conv1':
#         continue
#     cfg.defrost()
#     if idx > 13:
#         cfg.train.optim.max_iter = 50
#         lcp.prune_layer(
#             layer_name   = layer_name,
#             discard_rate = 0.6,
#         )
#     else:
#         cfg.train.optim.max_iter = 25
#         lcp.prune_layer(
#             layer_name   = layer_name,
#             discard_rate = 0.8,
#         )
#     cfg.freeze()
#     lcp_finetune.start_finetune(
#         criterion= lcp_criterion,  # 使用自定義損失函數
#     )
#     show_gpu_memory_usage()
#     print(f"剪枝後網路狀態 (第 {idx+1}, {layer_name} 層):")
#     show_network_status(lcp._net, lcp_finetune._prune_net)
#     lcp._prune_net = lcp_finetune._prune_net  # 更新剪枝網路
#     lcp.save_checkpoint_with_pruned_net(
#         log_path = './src/util/checkpoints-test2',
#         optimizer = optimizer,
#         model_name = f"lcp_finetune_{idx+1}_{layer_name}",
#         i_iter = cfg.train.optim.max_iter
#     )
#     lcp.debug_for_test_vision(
#         dataloader_train = dataloader_train,
#         img_normalization = img_normalization,
#         box_coder = box_coder,
#         cfg = cfg,
#         count = 2
#     )

##### The Global Finetune

In [ ]:
show_network_status(lcp._net, lcp._prune_net)

In [ ]:
global_finetune_iter = 9000
cfg.defrost()
cfg.train.optim.max_iter = 100
cfg.freeze()

In [ ]:
for iter in range( int(global_finetune_iter / cfg.train.optim.max_iter) ):
    print(f"Global Finetune Iteration: {iter+1}/{int(global_finetune_iter / cfg.train.optim.max_iter)}")
    lcp_finetune.start_finetune(
        criterion= lcp_criterion,  # 使用自定義損失函數
    )
    lcp.save_checkpoint_with_pruned_net(
        log_path = './src/util/checkpoints-test',
        optimizer = optimizer,
        model_name = f"lcp_finetune_{iter+1}_global",
        i_iter = cfg.train.optim.max_iter
    )
    lcp._prune_net = lcp_finetune._prune_net  # 更新剪枝網路
    lcp.debug_for_test_vision(
        dataloader_train = dataloader_train,
        img_normalization = img_normalization,
        box_coder = box_coder,
        cfg = cfg,
        count = 2
    )